In [11]:
import json

# read json
with open('covenants.json', 'r') as file:
    data = json.load(file)  

strings = []

for document in data:
    for result in document['annotations'][0]['result']:
        try:
            string = result['value']['text']
            label = result['value']['hypertextlabels'][0]
            label = ''.join(label.split())
            words = string.split()
            processed_words = []
            for i, word in enumerate(words):
                if i == 0:
                    processed_words.append(f"{word} B-{label}")
                else:
                    processed_words.append(f"{word} I-{label}")
    
            processed = "\n".join(processed_words)
            strings.append(processed)

        except KeyError:
            pass
            
output = "\n\n".join(strings)

output_file_path = 'output.txt'

with open(output_file_path, 'w') as file:
    file.write(output)

print(f"Processed string has been written to {output_file_path}")

Processed string has been written to output.txt
